In [ ]:
import os
import glob
import json
import requests
import csv
import time
from dotenv import load_dotenv

In [ ]:
load_dotenv()
VITO_CLIENT_ID = os.getenv("VITO_CLIENT_ID")
VITO_CLIENT_SECRET = os.getenv("VITO_CLIENT_SECRET")

if not VITO_CLIENT_ID or not VITO_CLIENT_SECRET:
    raise ValueError(
        "❌ .env 파일에 VITO_CLIENT_ID와 VITO_CLIENT_SECRET이 설정되지 않았습니다."
    )

In [ ]:
TARGET_FOLDERS = ["dataset/video1", "dataset/video2", "dataset/video3"]
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 처리할 파일 확장자
AUDIO_EXTENSIONS = [".mp4", ".mov", ".mkv", ".avi", ".m4a", ".mp3", ".wav"]

In [ ]:
def get_vito_token(client_id, client_secret):
    """VITO API 인증 토큰을 발급받습니다."""
    auth_url = "https://openapi.vito.ai/v1/authenticate"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {"client_id": client_id, "client_secret": client_secret}

    print("🔹 VITO 인증 토큰 발급 요청 중...")
    try:
        resp = requests.post(auth_url, headers=headers, data=data)
        resp.raise_for_status()  # 오류 발생 시 예외 처리
        token = resp.json()["access_token"]
        print("✅ 인증 토큰 발급 성공!")
        return token
    except requests.exceptions.RequestException as e:
        print(f"❌ VITO 인증 실패: {e}")
        raise


def format_time(seconds):
    """초를 'X분 Y초' 형태로 변환합니다."""
    if seconds is None:
        return "N/A"
    minutes, sec = divmod(int(seconds), 60)
    return f"{minutes}분 {sec}초" if minutes > 0 else f"{sec}초"


# --- 3. 메인 처리 로직 ---
def process_audio_files(jwt_token, target_folder, output_csv_path):
    """지정된 폴더의 오디오 파일들을 STT 처리하고 CSV에 추가합니다."""

    # 처리할 파일 목록 가져오기
    audio_files = []
    for ext in AUDIO_EXTENSIONS:
        audio_files.extend(glob.glob(os.path.join(target_folder, f"*{ext}")))

    if not audio_files:
        print(f"⚠️ '{target_folder}' 폴더에 처리할 파일이 없습니다. 건너뜁니다.")
        return

    total_files = len(audio_files)
    print(f"\n>>> '{target_folder}' 폴더의 총 {total_files}개 파일 처리 시작...")

    processed_times = []

    for i, file_path in enumerate(audio_files, start=1):
        filename = os.path.basename(file_path)
        print("\n" + "-" * 50)
        print(f"[ ▶️  파일 {i}/{total_files} ] {filename}")

        file_start_time = time.time()

        # VITO API 설정
        transcribe_url = "https://openapi.vito.ai/v1/transcribe"
        headers = {"Authorization": f"Bearer {jwt_token}"}
        config = {
            "use_diarization": True,
            "diarization": {"enable": True, "speaker_count": 2},
        }

        try:
            # 1. 전사 요청
            with open(file_path, "rb") as f:
                response = requests.post(
                    transcribe_url,
                    headers=headers,
                    data={"config": json.dumps(config)},
                    files={"file": f},
                )
            response.raise_for_status()
            job_id = response.json()["id"]
            print(f"[✔] 전사 요청 완료 - Job ID: {job_id}")

            # 2. 상태 확인
            status_url = f"{transcribe_url}/{job_id}"
            while True:
                print("... 분석 상태 확인 중 ...")
                status_resp = requests.get(status_url, headers=headers)
                status_resp.raise_for_status()
                status_data = status_resp.json()

                if status_data["status"] == "completed":
                    print("[✔] 분석 완료!")
                    segments = status_data.get("results", {}).get("utterances", [])
                    break
                elif status_data["status"] == "failed":
                    print(f"[❌] 전사 실패: {status_data.get('results')}")
                    segments = []
                    break
                time.sleep(5)

            # 3. CSV 저장
            if segments:
                with open(
                    output_csv_path, mode="a", newline="", encoding="utf-8-sig"
                ) as f:
                    writer = csv.writer(f)
                    for utt in segments:
                        original_spk = utt.get("spk", "")
                        # spk_0은 '피싱범(0)', 나머지는 '피해자(1)'로 분류
                        mapped_spk = "0" if original_spk == "spk_0" else "1"
                        writer.writerow(
                            [
                                filename,
                                utt.get("start_at"),
                                utt.get("duration"),
                                mapped_spk,
                                utt.get("msg"),
                                utt.get("lang"),
                            ]
                        )
                print(f"[📁] 저장 완료: {filename}")
            else:
                print(f"[!] 발화 내용이 없어 CSV에 저장하지 않습니다.")

        except requests.exceptions.RequestException as e:
            print(f"[❌] API 요청 중 오류 발생: {e}")
            # 토큰 만료(401) 시 프로그램 중단
            if e.response.status_code == 401:
                raise Exception(
                    "VITO 토큰이 만료되었거나 유효하지 않습니다. 재실행해주세요."
                )
            continue  # 다른 오류는 다음 파일로 계속 진행

        finally:
            file_end_time = time.time()
            time_taken = file_end_time - file_start_time
            processed_times.append(time_taken)
            avg_time = sum(processed_times) / len(processed_times)
            remaining_files = total_files - i
            eta = remaining_files * avg_time
            print(
                f"[📊 진행] 이번 파일: {format_time(time_taken)} | 평균: {format_time(avg_time)} | 예상 남은 시간: {format_time(eta)}"
            )


# --- 4. 스크립트 실행 ---
if __name__ == "__main__":

    # 1. VITO 토큰 발급
    vito_token = get_vito_token(VITO_CLIENT_ID, VITO_CLIENT_SECRET)

    # 2. 통합 CSV 파일 준비
    output_csv_path = os.path.join(OUTPUT_DIR, "vito_transcription_total.csv")
    csv_header = [
        "file_name",
        "start_at",
        "duration",
        "spk_label",
        "message",
        "language",
    ]

    # 파일이 없으면 헤더와 함께 새로 생성
    if not os.path.exists(output_csv_path):
        with open(output_csv_path, "w", newline="", encoding="utf-8-sig") as f:
            writer = csv.writer(f)
            writer.writerow(csv_header)
        print(f"\n결과를 저장할 '{output_csv_path}' 파일을 생성했습니다.")

    # 3. 설정된 모든 폴더에 대해 처리 실행
    total_start_time = time.time()
    for folder in TARGET_FOLDERS:
        process_audio_files(vito_token, folder, output_csv_path)

    total_end_time = time.time()
    total_elapsed = total_end_time - total_start_time

    print("\n" + "=" * 50)
    print(f"🎉 모든 폴더의 파일 처리가 완료되었습니다!")
    print(f"총 소요 시간: {format_time(total_elapsed)}")
    print(f"최종 결과는 '{output_csv_path}' 파일에서 확인하세요.")
    print("=" * 50)